# Kafka Streams

Kafka Streams is a client library for building mission-critical real-time applications and microservices, where the input and/or output data is stored in Kafka clusters. 

Kafka Streams combines the simplicity of writing and deploying standard Java and Scala applications on the client side with the benefits of Kafka's server-side cluster technology to make these applications highly scalable, elastic, fault-tolerant, distributed, and much more.

## Highlights 
https://kafka.apache.org/35/documentation/streams/core-concepts

Designed as a **simple and lightweight client library**, which can be easily embedded in any Java application and integrated with any existing packaging, deployment and operational tools that users have for their streaming applications.

i.e a dependency in a 
```xml
<dependency>
    <groupId>org.apache.kafka</groupId>
    <artifactId>kafka-streams</artifactId>
    <version>3.5.0</version>
</dependency>
```




Has **no external dependencies on systems other than Apache Kafka itself** as the internal messaging layer; notably, it uses Kafka's partitioning model to horizontally scale processing while maintaining strong ordering guarantees.

![](https://media.makeameme.org/created/keep-it-l0ywkb.jpg)
[Source](https://makeameme.org/meme/keep-it-l0ywkb)

Supports **fault-tolerant local state**, which enables very fast and efficient stateful operations like windowed joins and aggregations.



![](https://kafka.apache.org/0102/images/streams-architecture-states.jpg)

Supports **exactly-once processing** semantics to guarantee that each record will be processed once and only once even when there is a failure on either Streams clients or Kafka brokers in the middle of processing.

![](https://cdn.confluent.io/wp-content/uploads/kafka-topic.png)

https://www.confluent.io/blog/enabling-exactly-once-kafka-streams/

Employs **one-record-at-a-time processing** to achieve millisecond processing latency, and supports **event-time based windowing operations** with out-of-order arrival of records.

![](https://images.ctfassets.net/gt6dp23g0g38/y3dPJWV6inVi0KIWNk5Ic/5952ed5d7048e099a12ed57df173a39a/late-record-1.png)

https://developer.confluent.io/learn-kafka/kafka-streams/time-concepts/





Offers necessary stream processing primitives, along with a **high-level Streams DSL** and a **low-level Processor API**.

- High Level : https://kafka.apache.org/34/documentation/streams/developer-guide/dsl-api.html
- Processor API https://kafka.apache.org/34/documentation/streams/developer-guide/processor-api.html

# Use Cases

## The New York Times

![](https://kafka.apache.org/images/powered-by/NYT.jpg)

The New York Times uses Apache Kafka and the Kafka Streams to store and distribute, in real-time, published content to the various applications and systems that make it available to the readers.

[https://open.nytimes.com/publishing-with-apache-kafka-at-the-new-york-times-7f0e3b7d2077]

## Real Time Analytics

![](https://dzone.com/storage/temp/12275703-kafka-use-case.png)

Story: https://dzone.com/articles/real-time-stream-processing-with-apache-kafka-part-1

Code: https://github.com/hellosatish/microservice-patterns/tree/master/vehicle-tracker

# Example

# Wordcount

# Run Demo App
``` bash
# Start Zk
docker run -e KAFKA_ACTION=start-zk --network tap --ip 10.0.100.22  -p 2181:2181 --name kafkaZK -it tap:kafka
# Start Kafka Server
docker run -e KAFKA_ACTION=start-kafka --network tap --ip 10.0.100.23  -p 9092:9092 --name kafkaServer -it tap:kafka

# Create Topics (need to be created before start the stream)
docker exec -it kafkaServer kafka-topics.sh --bootstrap-server kafkaServer:9092 --create --topic streams-plaintext-input

docker exec -it kafkaServer kafka-topics.sh --bootstrap-server kafkaServer:9092 --create --topic streams-wordcount-output

# kafkaWordCountStream
docker exec -it kafkaServer /opt/kafka/bin/kafka-run-class.sh org.apache.kafka.streams.examples.wordcount.WordCountDemo

# Start a producer 
docker run --rm -e KAFKA_ACTION=producer -e KAFKA_TOPIC=streams-plaintext-input --network tap  -it tap:kafka

# Start consumer
docker run --rm -e KAFKA_ACTION=consumer -e KAFKA_TOPIC=streams-wordcount-output -e KAFKA_CONSUMER_PROPERTIES="--formatter kafka.tools.DefaultMessageFormatter --property print.key=true --property print.value=true --property key.deserializer=org.apache.kafka.common.serialization.StringDeserializer --property value.deserializer=org.apache.kafka.common.serialization.LongDeserializer" --network tap   -it tap:kafka
```

Messages:
- all streams lead to kafka
- hello kafka streams

# Behind the scenes

![](https://kafka.apache.org/34/images/streams-table-updates-01.png)

![](https://kafka.apache.org/34/images/streams-table-updates-02.png)

# Concepts

## Stream Processing Topology

- A stream is the most important abstraction provided by Kafka Streams: it represents an unbounded, continuously updating data set. A stream is an ordered, replayable, and fault-tolerant sequence of immutable data records, where a data record is defined as a key-value pair.
- A stream processing application is any program that makes use of the Kafka Streams library. It defines its computational logic through one or more processor topologies, where a processor topology is a graph of stream processors (nodes) that are connected by streams (edges).
- A stream processor is a node in the processor topology; it represents a processing step to transform data in streams by receiving one input record at a time from its upstream processors in the topology, applying its operation to it, and may subsequently produce one or more output records to its downstream processors.


![](https://kafka.apache.org/34/images/streams-architecture-topology.jpg)

Kafka Streams offers two ways to define the stream processing topology 

the Kafka Streams DSL provides the most common data transformation operations such as map, filter, join and aggregations out of the box; 

Processor API allows developers define and connect custom processors as well as to interact with state stores.

## Duality of Streams and Tables


When implementing stream processing use cases in practice, you typically need both streams and also databases. An example use case that is very common in practice is an e-commerce application that enriches an incoming stream of customer transactions with the latest customer information from a database table. In other words, streams are everywhere, but databases are everywhere, too.

**Stream as Table**: A stream can be considered a changelog of a table, where each data record in the stream captures a state change of the table. A stream is thus a table in disguise, and it can be easily turned into a "real" table by replaying the changelog from beginning to end to reconstruct the table. Similarly, in a more general analogy, aggregating data records in a stream - such as computing the total number of pageviews by user from a stream of pageview events - will return a table (here with the key and the value being the user and its corresponding pageview count, respectively).b

**Table as Stream**: A table can be considered a snapshot, at a point in time, of the latest value for each key in a stream (a stream's data records are key-value pairs). A table is thus a stream in disguise, and it can be easily turned into a "real" stream by iterating over each key-value entry in the table.

```java
// Serializers/deserializers (serde) for String and Long types
final Serde<String> stringSerde = Serdes.String();
final Serde<Long> longSerde = Serdes.Long();
 
// Construct a `KStream` from the input topic "streams-plaintext-input", where message values
// represent lines of text (for the sake of this example, we ignore whatever may be stored
// in the message keys).
KStream<String, String> textLines = builder.stream(
      "streams-plaintext-input",
      Consumed.with(stringSerde, stringSerde)
    );
 
KTable<String, Long> wordCounts = textLines
    // Split each text line, by whitespace, into words.
    .flatMapValues(value -> Arrays.asList(value.toLowerCase().split("\\W+")))
 
    // Group the text words as message keys
    .groupBy((key, value) -> value)
 
    // Count the occurrences of each word (message key).
    .count();
 
// Store the running counts as a changelog stream to the output topic.
wordCounts.toStream().to("streams-wordcount-output", Produced.with(Serdes.String(), Serdes.Long()));
```

# Stream and Tables: A Primer
https://www.confluent.io/blog/kafka-streams-tables-part-1-event-streaming/

## Event Record

**An event records the fact that “something happened” in the world**

- Event key: “Alice”
- Event value: “Has arrived in Rome”
- Event timestamp: “Dec. 3, 2019 at 9:06 a.m.”

## Event Stream

**An event stream records the history of what has happened in the world as a sequence of events**

This history is an ordered sequence or chain of events, so we know which event happened before another event to infer causality.

A stream thus represents both the past and the present: as we go from today to tomorrow—or from one millisecond to the next—new events are constantly being appended to the history.

### Example

_The sequence of moves in a chess match_

White moved the e2 pawn to e4, then Black moved the e7 pawn to e5

![](https://66.media.tumblr.com/tumblr_m8ok25dsch1r8gmlso1_500.gifv)

## Event Table

**A table represents the state of the world** at a particular point in time, typically “now.”

![](https://cdn.confluent.io/wp-content/uploads/streams-vs-tables-1.png)

| Stream | Table |
| ------ | ----- |
|A stream provides immutable data. It supports only inserting (appending) new events, whereas existing events cannot be changed. Streams are persistent, durable, and fault tolerant. Events in a stream can be keyed, and you can have many events for one key, like “all of Bob’s payments.” If you squint a bit, you could consider a stream to be like a table in a relational database (RDBMS) that has no unique key constraint and that is append only.| A table provides mutable data. New events—rows—can be inserted, and existing rows can be updated and deleted. Here, an event’s key aka row key identifies which row is being mutated. Like streams, tables are persistent, durable, and fault tolerant. Today, a table behaves much like an RDBMS materialized view because it is being changed automatically as soon as any of its input streams or tables change, rather than letting you directly run insert, update, or delete operations against it.|

|                                           | Stream |  Table    |
|-------------------------------------------|--------|-----------|
| First event with key bob arrives          | Insert | Insert    |
| Another event with key bob arrives        | Insert | Update    |
| Event with key bob and value null arrives | Insert | Delete    |
| Event with key null arrives               | Insert | _ignored_ |

![](https://cdn.confluent.io/wp-content/uploads/event-stream-1.gif)

# Writing App
https://docs.confluent.io/platform/current/streams/developer-guide/running-app.html

```bash
mvn archetype:generate \
    -DarchetypeGroupId=org.apache.kafka \
    -DarchetypeArtifactId=streams-quickstart-java \
    -DarchetypeVersion=3.5.0 \
    -DgroupId=streams.examples \
    -DartifactId=kafka-streams.examples \
    -Dversion=0.1 \
    -Dpackage=tap
```

## Let's make it run in Docker

- change props.put(StreamsConfig.BOOTSTRAP_SERVERS_CONFIG, "localhost:9092"); into kafkaServer:9092
- create topics
- build tap:kafkastream using build.sh
- run using docker run --network tap --rm -it tap:kafkastream class

### Pipe
``` bash
# Start Zk
docker run -e KAFKA_ACTION=start-zk --network tap --ip 10.0.100.22  -p 2181:2181 --name kafkaZK -it tap:kafka
# Start Kafka Server
docker run -e KAFKA_ACTION=start-kafka --network tap --ip 10.0.100.23  -p 9092:9092 --name kafkaServer -it tap:kafka

# Create Topics (need to be created before start the stream)
docker exec -it kafkaServer kafka-topics.sh --bootstrap-server kafkaServer:9092 --create --topic streams-plaintext-input
docker exec -it kafkaServer kafka-topics.sh --bootstrap-server kafkaServer:9092 --create --topic streams-pipe-output

# Run 
docker run --network tap --rm -it tap:kafkastream tap.Pipe
# Start a producer 
docker run --rm -e KAFKA_ACTION=producer -e KAFKA_TOPIC=streams-plaintext-input --network tap  -it tap:kafka

# Start consumer
docker run --rm -e KAFKA_ACTION=consumer -e KAFKA_TOPIC=streams-pipe-output --network tap -it tap:kafka
```

### LineSplit
```bash
# Start Zk
docker run -e KAFKA_ACTION=start-zk --network tap --ip 10.0.100.22  -p 2181:2181 --name kafkaZK -it tap:kafka
# Start Kafka Server
docker run -e KAFKA_ACTION=start-kafka --network tap --ip 10.0.100.23  -p 9092:9092 --name kafkaServer -it tap:kafka

# Create Topics (need to be created before start the stream)
docker exec -it kafkaServer kafka-topics.sh --bootstrap-server kafkaServer:9092 --create --topic streams-plaintext-input
docker exec -it kafkaServer kafka-topics.sh --bootstrap-server kafkaServer:9092 --create --topic streams-linesplit-output

# Run 
docker run --network tap --rm -it tap:kafkastream tap.LineSplit
# Start a producer 
docker run --rm -e KAFKA_ACTION=producer -e KAFKA_TOPIC=streams-plaintext-input --network tap  -it tap:kafka

# Start consumer
docker run --rm -e KAFKA_ACTION=consumer -e KAFKA_TOPIC=streams-linesplit-output --network tap -it tap:kafka
```

# Biblio
- https://blog.softwaremill.com/do-not-reinvent-the-wheel-use-kafka-connect-4bcabb143292
- https://dev.to/thegroo/kafka-connect-crash-course-1chd
- https://data-flair.training/blogs/kafka-connect/
- https://www.confluent.io/blog/simplest-useful-kafka-connect-data-pipeline-world-thereabouts-part-1/